In [ ]:
from engine import TradingEngine
import pandas as pd

bot = TradingEngine()

# Determine testing ranges
testing_ranges = [5, 10, 20, 50, 100, 200]
opt_results = []

# Download data once
df_base = bot.fetch_data(timeframe='1h', limit=1000)

for v in testing_ranges:
    # Do not manipulate original DataFrame
    df_temp = df_base.copy()
    
    # Run backtest with current window
    df_temp = bot.add_indicators(df_temp, sma_window=v)
    df_temp = bot.get_signals(df_temp, sma_window=v)
    res = bot.run_backtest(df_temp)
    
    # Store result
    opt_results.append({
        'Window': v,
        'Return %': res['total_return_pct'],
        'Final $': res['final_balance']
    })

# Convert to DataFrame for comparison
df_results = pd.DataFrame(opt_results)
print(df_results.sort_values(by='Return %', ascending=False))

In [ ]:
import matplotlib.pyplot as plt

# 1. Determine range (5, 10, 15... 300)
windows = range(5, 301, 5)
mReturns = []

df_base = bot.fetch_data(timeframe='1h', limit=2000) # More data = better statistics

for v in windows:
    df_temp = df_base.copy()
    df_temp = bot.add_indicators(df_temp, sma_window=v)
    df_temp = bot.get_signals(df_temp, sma_window=v)
    res = bot.run_backtest(df_temp)
    mReturns.append(res['total_return_pct'])

# 2. Plot optimisation curve
plt.figure(figsize=(12, 6))
plt.plot(windows, mReturns, marker='o', linestyle='-', color='b', markersize=4)
plt.axhline(0, color='red', linestyle='--') # Equilibrium line (0% profit)

plt.title('Parameter optimisation: SMA window vs Profitability')
plt.xlabel('SMA window size')
plt.ylabel('Total Return (%)')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
from engine import TradingEngine
import pandas as pd

symbol = 'BTC/USDT'  # Bitcoin
# symbol = 'LTC/USDT'  # Litecoin
# symbol = 'ETH/USDT'   # Ethereum
# symbol = 'ADA/USDT' # Cardano
# symbol = 'SOL/USDT' # Solana


bot = TradingEngine(symbol=symbol)  

# Download data once
df_base = bot.fetch_data(timeframe='1h', limit=1000)
df_base = bot.add_indicators(df_base, sma_window=10, ema_window=10, rsi_window=14, bb_window=20)

# Define parameter combinations to test
combinations = [
    {'sma_window':10, 'ema_window':None, 'use_macd':False, 'use_bbands':False},
    {'sma_window':10, 'ema_window':10, 'use_macd':False, 'use_bbands':False},
    {'sma_window':10, 'ema_window':10, 'use_macd':True, 'use_bbands':False},
    {'sma_window':10, 'ema_window':10, 'use_macd':True, 'use_bbands':True},
]

results = []

# Iterate over combinations
for params in combinations:
    df_temp = df_base.copy()
    df_temp = bot.get_signals(
        df_temp,
        sma_window=params['sma_window'],
        ema_window=params['ema_window'],
        use_macd=params['use_macd'],
        use_bbands=params['use_bbands'],
        rsi_overbought=60,
        rsi_oversold=50
    )
    # Run backtest
    res = bot.run_backtest(df_temp)
    results.append({
        **params,
        'Return %': res['total_return_pct'],
        'Final $': res['final_balance'],
        'Max Drawdown %': res['df']['drawdown'].min()*100
    })

# Turn into DataFrame for comparison
df_results = pd.DataFrame(results)
df_results.sort_values(by='Return %', ascending=False)
